# Modelo de Estimación de Edad para Good Seed

## Objetivo
Crear un modelo de visión artificial que estime la edad de personas a partir de fotografías para evitar la venta de alcohol a menores de edad.

## Contenido
1. Importación de librerías
2. Carga y exploración de datos
3. Preprocesamiento de imágenes
4. Creación del modelo
5. Entrenamiento
6. Evaluación
7. Conclusiones

## 1. Importación de Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

# TensorFlow y Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Configuración de semillas para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

## 2. Carga y Exploración de Datos

En esta sección cargamos el dataset de imágenes con edades etiquetadas. El dataset debería estar organizado en una carpeta con las imágenes y un archivo CSV con las etiquetas.

In [ ]:
# Definir rutas
DATA_PATH = Path('/datasets/faces')
LABELS_FILE = DATA_PATH / 'labels.csv'

# Nota: Este código asume que existe un dataset con la siguiente estructura:
# /datasets/faces/
#   ├── labels.csv (columnas: file_name, real_age)
#   └── final_files/
#       ├── imagen1.jpg
#       ├── imagen2.jpg
#       └── ...

# Para este ejemplo, crearemos datos sintéticos si no existe el dataset
if not DATA_PATH.exists():
    print("⚠️ Advertencia: No se encontró el dataset en /datasets/faces")
    print("Para ejecutar este notebook con datos reales:")
    print("1. Descarga el dataset de rostros con edades")
    print("2. Colócalo en /datasets/faces/")
    print("3. Asegúrate de tener un archivo labels.csv con columnas: file_name, real_age")
else:
    print(f"✓ Dataset encontrado en {DATA_PATH}")

In [ ]:
# Función para cargar y explorar los datos
def load_and_explore_data(labels_file):
    """
    Carga el archivo de etiquetas y muestra estadísticas básicas.
    
    Args:
        labels_file: Ruta al archivo CSV con las etiquetas
    
    Returns:
        DataFrame con las etiquetas
    """
    if not Path(labels_file).exists():
        print(f"⚠️ No se encontró el archivo {labels_file}")
        return None
    
    # Cargar datos
    df = pd.read_csv(labels_file)
    
    print("=" * 60)
    print("INFORMACIÓN DEL DATASET")
    print("=" * 60)
    print(f"\nTotal de imágenes: {len(df)}")
    print(f"\nColumnas: {df.columns.tolist()}")
    print(f"\nPrimeras filas:")
    print(df.head())
    
    print(f"\n\nEstadísticas de edad:")
    print(df['real_age'].describe())
    
    # Visualizar distribución de edades
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(df['real_age'], bins=30, edgecolor='black', alpha=0.7)
    plt.xlabel('Edad')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Edades en el Dataset')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.boxplot(df['real_age'])
    plt.ylabel('Edad')
    plt.title('Diagrama de Caja de Edades')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return df

# Intentar cargar los datos
if Path(LABELS_FILE).exists():
    df_labels = load_and_explore_data(LABELS_FILE)
else:
    print("\n📝 Ejecuta este notebook con un dataset real para continuar.")
    df_labels = None

## 3. Preprocesamiento de Imágenes

In [ ]:
# Configuración de hiperparámetros
IMG_SIZE = 224  # Tamaño de imagen para ResNet50
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.0001

# Función para crear generadores de datos
def create_data_generators(df, img_dir, val_split=0.2, test_split=0.1):
    """
    Crea generadores de datos para entrenamiento, validación y prueba.
    
    Args:
        df: DataFrame con file_name y real_age
        img_dir: Directorio con las imágenes
        val_split: Proporción para validación
        test_split: Proporción para prueba
    
    Returns:
        Tupla con (train_gen, val_gen, test_gen, train_df, val_df, test_df)
    """
    # Dividir datos
    train_val_df, test_df = train_test_split(df, test_size=test_split, random_state=42)
    train_df, val_df = train_test_split(train_val_df, test_size=val_split/(1-test_split), random_state=42)
    
    print(f"Conjunto de entrenamiento: {len(train_df)} imágenes")
    print(f"Conjunto de validación: {len(val_df)} imágenes")
    print(f"Conjunto de prueba: {len(test_df)} imágenes")
    
    # Generador de datos con aumento para entrenamiento
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        fill_mode='nearest'
    )
    
    # Generador para validación y prueba (solo reescalado)
    val_test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Crear generadores
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=img_dir,
        x_col='file_name',
        y_col='real_age',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='raw',
        shuffle=True
    )
    
    val_generator = val_test_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=img_dir,
        x_col='file_name',
        y_col='real_age',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='raw',
        shuffle=False
    )
    
    test_generator = val_test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=img_dir,
        x_col='file_name',
        y_col='real_age',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='raw',
        shuffle=False
    )
    
    return train_generator, val_generator, test_generator, train_df, val_df, test_df

# Crear generadores si hay datos
if df_labels is not None:
    IMG_DIR = DATA_PATH / 'final_files'
    train_gen, val_gen, test_gen, train_df, val_df, test_df = create_data_generators(
        df_labels, IMG_DIR
    )

## 4. Creación del Modelo

Usaremos transfer learning con ResNet50 pre-entrenado en ImageNet.

In [ ]:
def create_age_estimation_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    """
    Crea un modelo de estimación de edad usando ResNet50 pre-entrenado.
    
    Args:
        input_shape: Forma de las imágenes de entrada
    
    Returns:
        Modelo compilado de Keras
    """
    # Cargar ResNet50 pre-entrenado sin la capa superior
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Congelar las capas base inicialmente
    base_model.trainable = False
    
    # Agregar capas personalizadas
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    # Capa de salida: regresión para predecir edad
    output = Dense(1, activation='linear', name='age_output')(x)
    
    # Crear modelo
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compilar modelo
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='mae',  # Mean Absolute Error
        metrics=['mae', 'mse']
    )
    
    return model, base_model

# Crear el modelo
model, base_model = create_age_estimation_model()
print("\n" + "="*60)
print("RESUMEN DEL MODELO")
print("="*60)
model.summary()

## 5. Entrenamiento del Modelo

In [ ]:
# Configurar callbacks
callbacks = [
    ModelCheckpoint(
        'best_age_model.h5',
        monitor='val_mae',
        save_best_only=True,
        mode='min',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_mae',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Entrenar el modelo (solo si hay datos)
if df_labels is not None:
    print("\n" + "="*60)
    print("INICIANDO ENTRENAMIENTO - FASE 1")
    print("Entrenando solo las capas superiores (base congelada)")
    print("="*60 + "\n")
    
    history_phase1 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )
    
    # Fase 2: Fine-tuning - descongelar algunas capas del modelo base
    print("\n" + "="*60)
    print("INICIANDO ENTRENAMIENTO - FASE 2 (Fine-tuning)")
    print("Descongelando las últimas capas del modelo base")
    print("="*60 + "\n")
    
    # Descongelar las últimas 20 capas del modelo base
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    # Re-compilar con una tasa de aprendizaje más baja
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE/10),
        loss='mae',
        metrics=['mae', 'mse']
    )
    
    history_phase2 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS//2,
        callbacks=callbacks,
        verbose=1
    )
else:
    print("⚠️ No hay datos disponibles para entrenar el modelo.")
    print("Por favor, carga un dataset válido para continuar.")

In [ ]:
# Visualizar el historial de entrenamiento
if df_labels is not None:
    def plot_training_history(history1, history2=None):
        """
        Visualiza el historial de entrenamiento.
        """
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss
        axes[0].plot(history1.history['loss'], label='Train Loss (Phase 1)', linewidth=2)
        axes[0].plot(history1.history['val_loss'], label='Val Loss (Phase 1)', linewidth=2)
        if history2:
            offset = len(history1.history['loss'])
            axes[0].plot(range(offset, offset + len(history2.history['loss'])), 
                        history2.history['loss'], label='Train Loss (Phase 2)', linewidth=2)
            axes[0].plot(range(offset, offset + len(history2.history['val_loss'])), 
                        history2.history['val_loss'], label='Val Loss (Phase 2)', linewidth=2)
        axes[0].set_xlabel('Época')
        axes[0].set_ylabel('Loss (MAE)')
        axes[0].set_title('Pérdida durante el Entrenamiento')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # MAE
        axes[1].plot(history1.history['mae'], label='Train MAE (Phase 1)', linewidth=2)
        axes[1].plot(history1.history['val_mae'], label='Val MAE (Phase 1)', linewidth=2)
        if history2:
            offset = len(history1.history['mae'])
            axes[1].plot(range(offset, offset + len(history2.history['mae'])), 
                        history2.history['mae'], label='Train MAE (Phase 2)', linewidth=2)
            axes[1].plot(range(offset, offset + len(history2.history['val_mae'])), 
                        history2.history['val_mae'], label='Val MAE (Phase 2)', linewidth=2)
        axes[1].set_xlabel('Época')
        axes[1].set_ylabel('MAE')
        axes[1].set_title('Error Absoluto Medio durante el Entrenamiento')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    plot_training_history(history_phase1, history_phase2)

## 6. Evaluación del Modelo

In [ ]:
# Evaluar en conjunto de prueba
if df_labels is not None:
    print("\n" + "="*60)
    print("EVALUACIÓN EN CONJUNTO DE PRUEBA")
    print("="*60 + "\n")
    
    # Cargar el mejor modelo
    model.load_weights('best_age_model.h5')
    
    # Hacer predicciones
    predictions = model.predict(test_gen, verbose=1)
    predictions = predictions.flatten()
    
    # Obtener edades reales
    true_ages = test_df['real_age'].values
    
    # Calcular métricas
    mae = mean_absolute_error(true_ages, predictions)
    mse = mean_squared_error(true_ages, predictions)
    rmse = np.sqrt(mse)
    
    print(f"\nMétricas de Evaluación:")
    print(f"  MAE (Mean Absolute Error): {mae:.2f} años")
    print(f"  MSE (Mean Squared Error): {mse:.2f}")
    print(f"  RMSE (Root Mean Squared Error): {rmse:.2f} años")
    
    # Crear DataFrame de resultados
    results_df = pd.DataFrame({
        'Edad Real': true_ages,
        'Edad Predicha': predictions,
        'Error Absoluto': np.abs(true_ages - predictions)
    })
    
    print(f"\nEstadísticas del error:")
    print(results_df['Error Absoluto'].describe())
    
    # Visualizaciones
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Gráfico de dispersión: Real vs Predicho
    axes[0, 0].scatter(true_ages, predictions, alpha=0.5)
    axes[0, 0].plot([true_ages.min(), true_ages.max()], 
                    [true_ages.min(), true_ages.max()], 
                    'r--', linewidth=2, label='Predicción perfecta')
    axes[0, 0].set_xlabel('Edad Real')
    axes[0, 0].set_ylabel('Edad Predicha')
    axes[0, 0].set_title('Edad Real vs Edad Predicha')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Distribución de errores
    axes[0, 1].hist(results_df['Error Absoluto'], bins=30, edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(mae, color='r', linestyle='--', linewidth=2, label=f'MAE: {mae:.2f}')
    axes[0, 1].set_xlabel('Error Absoluto (años)')
    axes[0, 1].set_ylabel('Frecuencia')
    axes[0, 1].set_title('Distribución del Error Absoluto')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Residuos
    residuals = true_ages - predictions
    axes[1, 0].scatter(predictions, residuals, alpha=0.5)
    axes[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
    axes[1, 0].set_xlabel('Edad Predicha')
    axes[1, 0].set_ylabel('Residuos (Real - Predicho)')
    axes[1, 0].set_title('Gráfico de Residuos')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Análisis por rango de edad
    age_ranges = [(0, 18), (18, 25), (25, 35), (35, 50), (50, 100)]
    range_maes = []
    range_labels = []
    
    for start, end in age_ranges:
        mask = (true_ages >= start) & (true_ages < end)
        if mask.sum() > 0:
            range_mae = mean_absolute_error(true_ages[mask], predictions[mask])
            range_maes.append(range_mae)
            range_labels.append(f'{start}-{end}')
    
    axes[1, 1].bar(range_labels, range_maes, alpha=0.7, edgecolor='black')
    axes[1, 1].set_xlabel('Rango de Edad')
    axes[1, 1].set_ylabel('MAE (años)')
    axes[1, 1].set_title('MAE por Rango de Edad')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Análisis específico para menores de edad
    print("\n" + "="*60)
    print("ANÁLISIS PARA DETECCIÓN DE MENORES DE EDAD")
    print("="*60)
    
    # Calcular métricas para menores de 18 años
    minors_mask = true_ages < 18
    if minors_mask.sum() > 0:
        minors_mae = mean_absolute_error(true_ages[minors_mask], predictions[minors_mask])
        print(f"\nMAE para menores de 18 años: {minors_mae:.2f} años")
        
        # Análisis de clasificación: ¿Identificamos correctamente a menores?
        true_minors = true_ages < 18
        pred_minors = predictions < 18
        
        # Matriz de confusión simple
        tp = np.sum(true_minors & pred_minors)  # Verdaderos positivos
        tn = np.sum(~true_minors & ~pred_minors)  # Verdaderos negativos
        fp = np.sum(~true_minors & pred_minors)  # Falsos positivos
        fn = np.sum(true_minors & ~pred_minors)  # Falsos negativos
        
        print(f"\nClasificación binaria (menor/mayor de edad):")
        print(f"  Verdaderos Positivos (menor correctamente identificado): {tp}")
        print(f"  Verdaderos Negativos (mayor correctamente identificado): {tn}")
        print(f"  Falsos Positivos (mayor clasificado como menor): {fp}")
        print(f"  Falsos Negativos (menor clasificado como mayor): {fn}")
        
        if (tp + fn) > 0:
            recall = tp / (tp + fn)
            print(f"  \n⚠️ RECALL (Sensibilidad): {recall:.2%}")
            print(f"     → Porcentaje de menores correctamente identificados")
        
        if (tp + fp) > 0:
            precision = tp / (tp + fp)
            print(f"  Precisión: {precision:.2%}")
            print(f"     → De los clasificados como menores, cuántos realmente lo son")
    else:
        print("\n⚠️ No hay menores de edad en el conjunto de prueba.")

## 7. Visualización de Predicciones en Imágenes de Muestra

In [ ]:
# Mostrar algunas predicciones con imágenes
if df_labels is not None:
    from tensorflow.keras.preprocessing import image
    import random
    
    # Seleccionar algunas imágenes al azar del conjunto de prueba
    sample_indices = random.sample(range(len(test_df)), min(9, len(test_df)))
    
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.flatten()
    
    for idx, sample_idx in enumerate(sample_indices):
        # Cargar imagen
        img_name = test_df.iloc[sample_idx]['file_name']
        img_path = IMG_DIR / img_name
        
        img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        
        # Hacer predicción
        pred_age = model.predict(img_array, verbose=0)[0][0]
        true_age = test_df.iloc[sample_idx]['real_age']
        
        # Mostrar imagen con predicción
        axes[idx].imshow(img)
        axes[idx].axis('off')
        
        # Color del título: verde si la diferencia es pequeña, rojo si es grande
        error = abs(pred_age - true_age)
        color = 'green' if error < 5 else 'orange' if error < 10 else 'red'
        
        axes[idx].set_title(
            f'Real: {true_age:.0f} años\nPredicho: {pred_age:.1f} años\nError: {error:.1f}',
            fontsize=10,
            color=color,
            weight='bold'
        )
    
    plt.tight_layout()
    plt.suptitle('Ejemplos de Predicciones del Modelo', fontsize=16, y=1.00)
    plt.show()

## 8. Conclusiones y Recomendaciones

### Conclusiones

1. **Modelo Desarrollado**: Se ha creado un modelo de visión artificial basado en ResNet50 con transfer learning para estimar la edad a partir de fotografías.

2. **Métricas de Rendimiento**: El modelo ha sido evaluado utilizando MAE (Error Absoluto Medio) como métrica principal, que indica el error promedio en años.

3. **Aplicación Práctica**: Para el caso de uso de Good Seed (prevención de venta de alcohol a menores), se ha analizado específicamente:
   - La precisión del modelo para identificar menores de 18 años
   - El recall (sensibilidad) para evitar que menores sean clasificados como mayores
   - El análisis de falsos negativos (crítico en este contexto)

### Recomendaciones

1. **Para Implementación en Producción**:
   - Establecer un umbral de decisión conservador (por ejemplo, clasificar como menor si la edad predicha es < 21 años para tener un margen de seguridad)
   - Implementar verificación manual cuando la predicción esté cerca del límite de edad
   - Considerar el recall como métrica más importante que la precisión (mejor prevenir falsos negativos)

2. **Mejoras Futuras**:
   - Aumentar el dataset con más imágenes, especialmente en el rango de 16-20 años
   - Explorar arquitecturas más modernas (EfficientNet, Vision Transformers)
   - Implementar técnicas de ensemble con múltiples modelos
   - Considerar factores adicionales como calidad de imagen, iluminación, ángulo

3. **Consideraciones Éticas**:
   - Asegurar que el modelo no tenga sesgos por etnia, género u otras características
   - Implementar salvaguardas de privacidad para las imágenes capturadas
   - Establecer políticas claras sobre el uso y almacenamiento de datos
   - Mantener siempre una opción de verificación manual

4. **Monitoreo Continuo**:
   - Implementar logging de predicciones para análisis posterior
   - Reentrenar el modelo periódicamente con nuevos datos
   - Monitorear métricas de rendimiento en producción
   - Recopilar feedback de empleados sobre casos difíciles

## Siguiente Paso: Guardar el Modelo

Para usar el modelo en producción, guárdalo en diferentes formatos:

In [ ]:
# Guardar el modelo completo
if df_labels is not None:
    # Formato H5
    model.save('age_estimation_model_final.h5')
    print("✓ Modelo guardado en formato H5: age_estimation_model_final.h5")
    
    # Formato SavedModel (para TensorFlow Serving)
    model.save('age_estimation_model_savedmodel')
    print("✓ Modelo guardado en formato SavedModel: age_estimation_model_savedmodel/")
    
    print("\n📦 El modelo está listo para ser desplegado en producción.")